In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('../data/res_mapping.csv', nrows=5000)
data['res_dt']=pd.to_datetime(data['res_dt'], format="%d.%m.%y %H:%M")
data['dep_soll']=pd.to_datetime(data['dep_soll'], format="%d.%m.%y %H:%M")
data['diff']=abs((data['res_dt']-data['dep_soll']).dt.total_seconds())
data

In [ ]:
tokens = list(set(np.hstack([data['bp_from'], data['bp_to']])))

In [ ]:
train_mappings=pd.read_csv('../data/train_mapping.csv')
train_mappings

In [ ]:
block_mappings=pd.read_csv('../data/block_mapping.csv')
block_mappings

In [ ]:
for index, row in data.iterrows():
    data.at[index, 'bp_from_token'] = tokens.index(row['bp_from']) / len(tokens)
    data.at[index, 'bp_to_token'] = tokens.index(row['bp_to']) / len(tokens)
    data.at[index, 'res_count'] = len(data.loc[np.logical_and(data['train_nr'] == row['train_nr'], data['dep_soll'] == row['dep_soll'])])
    #data.at[index, 'res_count'] = len(data.loc[np.logical_and(data['train_nr'] == row['train_nr'], data['dep_soll'] == row['dep_soll'])])
    
    block_nr = train_mappings.loc[train_mappings['Zug'] == row['train_nr']].iloc[0, 3]
    found_block = block_mappings.loc[block_mappings['Block Bezeichnung in Jahresformation Fpl-2022'] == block_nr]
    
    if len(found_block) > 0:
        data.at[index, 'max_res_count'] = found_block.iloc[0, 1] 
    else:
        data.at[index, 'max_res_count'] = 0

In [ ]:
counts = data['max_res_count'] - data['res_count']
len(counts.where(counts <= 0).dropna())

In [ ]:
data.to_csv('../data/output.csv', columns = ['diff', 'bp_from_token', 'bp_to_token'], index=False)